In [2]:
import pandas as pd
import numpy as np
pred_df = pd.read_csv("/Users/christianhellum/Cand. Merc./Data-Science-Project/data_science_project/Beat the bookies/data/df_preprocessed.csv", index_col=0)

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix

X = pred_df.drop(['FTR', 'Date'], axis=1)
Y = pred_df['FTR']

X_test = pred_df.drop(['FTR', 'Date'], axis=1)
y_test = pred_df['FTR']

In [4]:
import keras
from keras.layers import Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping
import tensorflow as tf
import matplotlib.pyplot as plt
import os

2024-06-18 08:33:18.690408: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
def transform_results(results):
    transformed = []
    for i in range(len(results)):
        if results[i] == 'H':
            transformed.append(0)
        elif results[i] == 'A':
            transformed.append(2)
        else:
            transformed.append(1)
    return np.array(transformed)
            
Y = transform_results(Y)
y_test = transform_results(y_test)

In [6]:
scaler = StandardScaler()

X = scaler.fit_transform(X)
X_test_scaled = scaler.fit_transform(X_test)

In [7]:
X_train_scaled = X[:-160]
y_train = Y[:-160]
X_test_scaled = X[-160:]
y_test = y_test[-160:]

In [8]:
print('Number of matches in training data:', X_train_scaled.shape[0])
print('Number of matches in test data:', X_test_scaled.shape[0])
print('Number of features:', X_train_scaled.shape[1])

Number of matches in training data: 1700
Number of matches in test data: 160
Number of features: 33


In [29]:
# input dimension is number of features
input_dim = X_train_scaled.shape[1]

activation_func = 'relu'
kernel_init = 'glorot_normal'
batch_size = 4

model = keras.Sequential([
    Dense(64, input_shape=(input_dim,), activation=activation_func),
    Dropout(0.3),
    Dense(32),
    Dropout(0.7),
    Dense(16),
    Dense(3, activation='softmax')
])

/Users/christianhellum/Library/Caches/pypoetry/virtualenvs/data-science-project-PFmYB0_u-py3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [23]:
from keras.optimizers import Adam, SGD

In [30]:
es = EarlyStopping(monitor='loss', patience=3, verbose=1)

learning_rate = 0.001


opt = Adam(learning_rate=learning_rate)

# Compile your model with the optimizer
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train_scaled, y_train, batch_size=batch_size, callbacks=[es], epochs=500, verbose=0)

Epoch 8: early stopping


In [31]:
train_loss, train_acc = model.evaluate(X_train_scaled, y_train)
test_loss, test_acc = model.evaluate(X_test_scaled, y_test)
print('Training loss:', train_loss)
print('Training accuracy:', train_acc)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5277 - loss: 0.9680
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5423 - loss: 0.9829 
Training loss: 0.9627589583396912
Training accuracy: 0.5394117832183838
Test loss: 0.9665796160697937
Test accuracy: 0.550000011920929


In [13]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
y_preds = model.predict(X_test_scaled)
y_pred_argmax = [np.argmax(i) for i in y_preds]
print(classification_report(y_test, y_pred_argmax))

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
              precision    recall  f1-score   support

           0       0.56      0.92      0.69        73
           1       0.00      0.00      0.00        38
           2       0.65      0.53      0.58        49

    accuracy                           0.58       160
   macro avg       0.40      0.48      0.43       160
weighted avg       0.45      0.58      0.50       160



/Users/christianhellum/Library/Caches/pypoetry/virtualenvs/data-science-project-PFmYB0_u-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/christianhellum/Library/Caches/pypoetry/virtualenvs/data-science-project-PFmYB0_u-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/christianhellum/Library/Caches/pypoetry/virtualenvs/data-science-project-PFmYB0_u-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precis

In [ ]:
funds = 100
wager = 10
favourites = 0
no_bets = 0
min_diff = 0.03

y_preds = model.predict(X_test_scaled)

for i in range(len(X_test_scaled)):
    prediction = np.argmax(y_preds[i])
    print('\nPrediction', prediction)
    print('Actual', y_test[i])
    print('Favourite', np.argmin([X_test['B365H'][i], X_test['B365D'][i], 
                                  X_test['B365A'][i]]))
    print('Prediction proba', y_preds[i])
    print('Home, Draw and Away odds', X_test['B365H'][i],       
          X_test['B365D'][i], X_test['B365A'][i])
    
    if prediction == 0:
        odds_diff = y_preds[i][prediction] - (1/X_test['B365H'][i])
        if odds_diff > min_diff:
            if prediction == np.argmin([X_test['B365H'][i], X_test['B365D'][i], 
                                  X_test['B365A'][i]]):
                favourites +=1
                
            if  prediction == y_test[i]:
                funds += (wager * X_test['B365H'][i]) - wager
            else:
                funds -= wager
        else:
            no_bets +=1
            
    elif prediction == 1:
        odds_diff = y_preds[i][prediction] - (1/X_test['B365D'][i])
        if odds_diff > min_diff:
            if prediction == np.argmin([X_test['B365H'][i], X_test['B365D'][i], 
                                  X_test['B365A'][i]]):
                favourites +=1
            if  prediction == y_test[i]:
                funds +=( wager * X_test['B365D'][i]) - wager
            else:
                funds -= wager
        else:
            no_bets +=1
    else:
        odds_diff = y_preds[i][prediction] - (1/X_test['B365A'][i])
        if odds_diff >  min_diff:
            if prediction == np.argmin([X_test['B365H'][i], X_test['B365D'][i], 
                                  X_test['B365A'][i]]):
                favourites +=1
            if  prediction == y_test[i]:
                funds += (wager * X_test['B365A'][i]) - wager
            else:
                funds -= wager
        else:
            no_bets +=1
   
    print('Funds', funds)
    
print(f'Betted on favourite {favourites} times out of {len(X_test)} matches.')
print(f'No bet placed {no_bets} times')